In [ ]:
%pip install model_deploy-0.0.0-py3-none-any.whl

In [ ]:
%pip install -e ..
%restart_python

In [ ]:
from pathlib import Path
import sys
sys.path.append(str(Path.cwd().parent / 'src'))

In [ ]:

import mlflow
from loguru import logger
from pyspark.sql import SparkSession
from pyspark.dbutils import DBUtils
from importlib.metadata import version
from mlflow.models import infer_signature

import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

from model_deploy.config import ProjectConfig, Tags
from model_deploy.models.basic_model import BasicModel
from pyspark.sql import SparkSession
from pyspark.dbutils import DBUtils

spark = SparkSession.builder.getOrCreate()
dbutils = DBUtils(spark)

In [ ]:
%sql
CREATE TABLE IF NOT EXISTS mlops_dev.model_test.training_control (
    allow_training BOOLEAN,
    updated_on TIMESTAMP
);

INSERT INTO mlops_dev.model_test.training_control VALUES (true, current_timestamp());


In [ ]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

# -----------------------------------------------------------
# PATCH BASICMODEL FOR SKLEARN DATASET
# -----------------------------------------------------------
def patch_basicmodel_for_sklearn():
    """Patch BasicModel.load_data() and BasicModel.prepare_features() 
       to use sklearn dataset instead of Delta tables.
    """

    def load_data(self):
                # ----------------------------------------------------
        # 🔐 TRAINING CONTROL CHECK
        # ----------------------------------------------------
        logger.info("🔍 Checking training control flag in Lakehouse...")

        flag = spark.sql("""
            SELECT allow_training
            FROM mlops_dev.model_test.training_control
            ORDER BY updated_on DESC
            LIMIT 1
        """).first()[0]

        if not flag:
            logger.warning("⛔ Training disabled via training_control table. Skipping training.")
            # Tell pipeline to skip
            dbutils.jobs.taskValues.set("training_skipped", "TRUE")
            sys.exit(0)

        logger.info("✅ Training allowed — proceeding with dataset loading.")

        # ----------------------------------------------------
        # LOAD SKLEARN DATASET
        # ----------------------------------------------------
        logger.info("🔄 Loading sklearn breast cancer dataset...")

        data = load_breast_cancer()
        df = pd.DataFrame(data.data, columns=data.feature_names)
        df["target"] = data.target

        # ALL numeric features
        self.num_features = list(df.columns)
        self.num_features.remove("target")

        self.cat_features = []   # No categorical features

        # Train/Test split
        train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

        self.train_set = train_df
        self.test_set = test_df

        self.X_train = train_df[self.num_features]
        self.y_train = train_df["target"]
        self.X_test = test_df[self.num_features]
        self.y_test = test_df["target"]

        # Needed for model_improved()
        self.eval_data = test_df.copy()

        logger.info("✅ sklearn dataset loaded successfully.")

    def prepare_features(self):
        logger.info("🔄 No preprocessing needed for sklearn dataset.")
        from lightgbm import LGBMClassifier
        self.pipeline = LGBMClassifier(**self.parameters)
        logger.info("✅ Pipeline ready using LGBMClassifier.")

    def log_model(self):
        logger.info("📦 Logging model + parameters + metrics to MLflow...")

        # mlflow.set_experiment(self.experiment_name)
        mlflow.set_experiment("/Shared/mlops_exp")

        with mlflow.start_run(run_name="basic-lgbm", tags=self.tags) as run:

            # ⭐ save run_id for register_model()
            self.run_id = run.info.run_id
            dbutils.jobs.taskValues.set("candidate_run_id", self.run_id)
            # --- parameters ---
            mlflow.log_params(self.config.parameters)

            # --- metrics ---
            y_pred = self.pipeline.predict(self.X_test)

            self.metrics = {
                "f1_score": float(f1_score(self.y_test, y_pred)),
                "accuracy": float(accuracy_score(self.y_test, y_pred)),
                "precision": float(precision_score(self.y_test, y_pred)),
                "recall": float(recall_score(self.y_test, y_pred)),
            }

            mlflow.log_metrics(self.metrics)

            # --- model artifact ---
            logger.info("📁 Logging sklearn LightGBM model...")
            signature = infer_signature(self.X_train, self.pipeline.predict(self.X_train))

            self.model_info = mlflow.sklearn.log_model(
                sk_model=self.pipeline,
                artifact_path="model",
                signature=signature,
                input_example=self.X_train.iloc[0:1],
            )

        logger.info(f"✅ MLflow logging completed. Run ID: {self.run_id}, Metrics: {self.metrics}")


        # Patch methods
    BasicModel.load_data = load_data
    BasicModel.prepare_features = prepare_features
    BasicModel.log_model = log_model

In [ ]:

# -----------------------------------------------------------
# MANUAL RUN (NO ARGPARSE NEEDED)
# -----------------------------------------------------------

env = "dev"  # You can change manually: dev / acc / prd
config_path = "model_config_deploy.yml"

spark = SparkSession.builder.getOrCreate()
dbutils = DBUtils(spark)

# Manual tags for logging
tags = Tags(
    git_sha="manual_run",
    branch="manual_run",
    job_run_id="manual_run"#spark.conf.get("spark.databricks.job.id", "unknown_job_id")
)

config = ProjectConfig.from_yaml(config_path=config_path, env=env)

In [ ]:

# Patch BEFORE running training
patch_basicmodel_for_sklearn()
print(config)
basic_model = BasicModel(config=config, tags=tags, spark=spark)

basic_model.load_data()
basic_model.prepare_features()

basic_model.train()
basic_model.log_model()

model_improved = basic_model.model_improved() # Ensure the model alias exists before calling model_improved, or handle the case where it does not.



In [ ]:
from mlflow import MlflowClient, register_model
client = MlflowClient()
print("Tracking URI:", mlflow.get_tracking_uri())

# Check model visibility
try:
    print("Registered model:", client.get_registered_model("mlops_dev.model_test.model_deploy"))
except Exception as e:
    print("Registered model not found or error:", e)

# Check alias (if supported)
try:
    mv = client.get_model_version_by_alias("mlops_dev.model_test.model_deploy", "latest-model")
    print("Alias latest-model ->", mv.version)
except Exception as e:
    print("Alias lookup error:", e)


In [ ]:
# # -----------------------------------------------------------
# # If model is better → register + wrap
# # -----------------------------------------------------------

# if model_improved:
#     # Register version
#     new_version = basic_model.register_model()

#     pkg_version = version("model_deploy")  # your package name

#     pyfunc_model_name = (
#         f"{config.catalog_name}.{config.schema_name}.model_deploy_custom"
#     )

#     # code_paths = [
#     #     f"./artifacts/.internal/model_deploy-{pkg_version}-py3-none-any.whl"
#     # ]
#     import model_deploy, os
#     code_dir = os.path.dirname(model_deploy.__file__)
#     wrapper = MarvelModelWrapper()

#     latest_version = wrapper.log_register_model(
#         wrapped_model_uri=f"{basic_model.model_info.model_uri}",
#         pyfunc_model_name=pyfunc_model_name,
#         experiment_name=config.experiment_name_custom,
#         input_example=basic_model.X_test[0:1],
#         tags=tags,
#         code_paths=[code_dir],
#     )

#     logger.info(f"🟢 New model registered → version {latest_version}")

#     dbutils.jobs.taskValues.set(key="model_version", value=latest_version)
#     dbutils.jobs.taskValues.set(key="model_updated", value=1)

# else:
#     logger.info("🔴 Model did NOT improve → not registering")
#     dbutils.jobs.taskValues.set(key="model_updated", value=0)
